In [1]:
import os

In [8]:
%pwd

'd:\\Chicken-Disease-Classification'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Chicken-Disease-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir: Path

In [7]:
from cnnClassifier.components import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [10]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH


class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_igestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [11]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_igestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-27 22:54:39,144: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-27 22:54:39,147: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-27 22:54:39,150: INFO: common: created directory at: artifacts]
[2024-02-27 22:54:39,153: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-27 22:58:02,840: INFO: 174730537: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23467027
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "0dff4fcd5b560a355beacf7b63e946e2143131d871fa2577664721c5a15c09b7"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4886:166709:51FF05:6AFBEF:65DE090F
Accept-Ranges: bytes
Date: Tue, 27 Feb 2024 17:22:55 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10226-MAA
X-Cache: H